In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

fake_folder = 'ai_images'
real_folder = 'real_images'

fake_images = [os.path.join(fake_folder, f) for f in os.listdir(fake_folder)]
real_images = [os.path.join(real_folder, f) for f in os.listdir(real_folder)]

fake_labels = ['FAKE'] * len(fake_images)
real_labels = ['REAL'] * len(real_images)


df2 = pd.DataFrame({
    'filepath': fake_images + real_images,
    'label': fake_labels + real_labels
})

df_combined = pd.concat([df, df2], ignore_index=True)
df_combined = df_combined.drop(columns=['Unnamed: 0','file_name'], errors='ignore')



In [ ]:
df_combined['label'] = df_combined['label'].map({
    0: 'REAL',       
    1: 'FAKE',        
    'REAL': 'REAL',  
    'FAKE': 'FAKE'    
})


print(df_combined['label'].unique())  #['REAL' 'FAKE']
df_combined.label.value_counts()

['FAKE' 'REAL']


label
FAKE    63975
REAL    63975
Name: count, dtype: int64

In [3]:
train_df, val_df = train_test_split(df_combined, test_size=0.2, random_state=42)
datagen = ImageDataGenerator(
    rescale=1./255,           
    rotation_range=40,      
    width_shift_range=0.2,    
    height_shift_range=0.2,   
    shear_range=0.2,          
    zoom_range=0.2,           
    horizontal_flip=True,     
    fill_mode='nearest'      
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',  
    y_col='label',    
    target_size=(224, 224),
    batch_size=32, # ajustable (better gpu > 64)
    class_mode='binary'  
)

# validation
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32, # ajustable (better gpu > 64)
    class_mode='binary'
)

Found 102360 validated image filenames belonging to 2 classes.
Found 25590 validated image filenames belonging to 2 classes.


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) # check gpu available


Num GPUs Available:  1


In [4]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16') # reduce calculation size from float32 to 16
mixed_precision.set_global_policy(policy)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Callbacks
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Model using MobileNetV2
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,               # Exclude the top layer as we will add our own classifier
    weights='imagenet'               # Load pre-trained weights from ImageNet
)

base_model.trainable = False  # Freeze base model layers for transfer learning

# Model architecture
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs)  # Apply MobileNetV2 as the base feature extractor
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling to reduce the output size
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.002))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.002))(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,  # Assuming train_ds is the dataset
    validation_data=val_generator,  # Assuming val_ds is the validation dataset
    epochs=200,
    callbacks=[early_stop, lr_schedule],
)

# Plot Accuracy & Loss
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
model.save('MobileNetV2_Full.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model.save_weights('MobileNetV2_Full_weight.h5')